In [1]:
# -*- coding: utf-8 -*-

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import tensorflow as tf
import numpy as np
import time

from data_loader import movie_data
data = movie_data()

def initialize_session():
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.4
    return tf.Session(config=config)



/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
##################################################
BATCH_SIZE = 16
class_size = 4
learning_rate = 0.0001   # Learning rate
use_clip = True         # Gradient clipping 쓸지 여부
keep_prob = 1.0
##################################################
from genre_model import Model
model = Model(use_clip=use_clip, class_size=class_size, learning_rate=learning_rate)

sess = initialize_session()
sess.run(tf.global_variables_initializer())

def test_model():
    num_it = 10
    test_loss, test_cnt, test_right = 0, 0, .0

    for _ in range(num_it):
        test_x, test_label = data.get_test(BATCH_SIZE)
        loss, out = sess.run([model.loss, model.y_pred],
                              feed_dict={model.x_image: test_x, model.y_label: test_label, model.keep_prob : 1.0})
        test_loss += loss
        test_cnt += 1
        for i, o in enumerate(out):
            if o == test_label[i]:
                test_right += 1
    print(" * test loss: {:.3f} | acc: {:.3f}\n".format(test_loss / test_cnt, test_right / test_cnt / BATCH_SIZE))



In [3]:
# 0: neg, 1: pos
avg_loss, it_cnt, same = 0, 0, .0
it_log, it_test, it_save, it_sample = 10, 100, 1000, 100
start_time = time.time()

for it in range(0, 5000):
    train_x, label = data.get_train(BATCH_SIZE)
    loss, _, out = sess.run([model.loss, model.update, model.y_pred],
                            feed_dict={model.x_image: train_x, model.y_label: label, model.keep_prob : 0.75})

    for i, o in enumerate(out):
        if o == label[i]:
            same += 1

    avg_loss += loss
    it_cnt += 1
    if it % it_log == 0 and it:
        print(" it: {:4d} | loss: {:.3f} | acc: {:.3f} - {:.2f}s".format(
            it, avg_loss / it_cnt, same/BATCH_SIZE/it_log, time.time() - start_time))
        avg_loss, it_cnt, same = 0, 0, .0

#     if it % it_test == 0 and it > 0:
#         test_model()
    if it % it_save == 0 and it > 0:
        model.save(sess)


ResourceExhaustedError: OOM when allocating tensor with shape[16,256,134,91] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, wc1/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: gradients/add_grad/Shape/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_137_gradients/add_grad/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add r

In [ ]:
type(label)